In [5]:
!pip install ultralytics opencv-python pandas

In [6]:
import cv2
import pandas as pd
import torch
from ultralytics import YOLO

In [13]:
# Load the YOLOv8 model (pre-trained on COCO dataset)
model = YOLO('yolov8n.pt')  # Using the nano version for speed; can use yolov8s.pt for better accuracy

video_path = '/content/drive/MyDrive/dataset_video.mp4'
cap = cv2.VideoCapture(video_path)
csv_file_path = "/content/crowd_detection_log.csv"

In [14]:
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

In [15]:
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define codec and create VideoWriter object to save output
output_video_path = "/content/crowd_detection_output.mp4"  # Save in Colab's default directory
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

In [16]:
crowd_data = [] # Stores timestamps where a crowd event is detected
crowd_threshold = 5  # Adjust this based on what defines a "crowd"

frame_count = 0

In [17]:
# Process each frame
while True:
    ret, frame = cap.read()
    if not ret:
        break  # Exit when video ends

    frame_count += 1
    timestamp = frame_count / fps  # Convert frame count to time in seconds

    # Run object detection on frame
    results = model(frame)

    # Track detected people count
    person_count = 0

    # Draw detections on frame
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            conf = box.conf[0]  # Confidence score
            cls = int(box.cls[0])  # Class index

            # Draw bounding box if detected object is a person (class 0 in COCO dataset)
            if cls == 0:
                person_count += 1
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                cv2.putText(frame, f"Person {conf:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # If crowd size exceeds threshold, log the event
    if person_count >= crowd_threshold:
        crowd_data.append([timestamp, frame_count, person_count])

    # Write modified frame to output video
    video_writer.write(frame)

# Release resources
cap.release()
video_writer.release()

# Save crowd log to a file
df = pd.DataFrame(crowd_data, columns=["Timestamp (s)", "Frame", "Person Count"])
df.to_csv(csv_file_path, index=False)

print("Processed video saved at:", output_video_path)
print("Crowd log saved at:", csv_file_path)



0: 384x640 37 persons, 2 birds, 10.0ms
Speed: 2.7ms preprocess, 10.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 persons, 3 birds, 9.2ms
Speed: 3.7ms preprocess, 9.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 35 persons, 3 birds, 14.6ms
Speed: 5.0ms preprocess, 14.6ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 37 persons, 3 birds, 12.6ms
Speed: 3.2ms preprocess, 12.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 37 persons, 2 birds, 9.8ms
Speed: 3.4ms preprocess, 9.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 persons, 2 birds, 9.6ms
Speed: 3.4ms preprocess, 9.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 36 persons, 2 birds, 10.0ms
Speed: 3.3ms preprocess, 10.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 36 persons, 2 birds, 10.5ms
Speed: 

In [18]:
import shutil

drive_video_path = "/content/drive/My Drive/crowd_detection_output.mp4"
drive_csv_path = "/content/drive/My Drive/crowd_detection_log.csv"

shutil.copy(output_video_path, drive_video_path)
shutil.copy(csv_file_path, drive_csv_path)

print("Files moved to Google Drive successfully!")


Files moved to Google Drive successfully!


In [19]:
import shutil

source = "/content/crowd_detection_output.mp4"
destination = "/content/drive/My Drive/crowd_detection_output.mp4"

shutil.copy(source, destination)
print("Video successfully moved to Google Drive!")

Video successfully moved to Google Drive!
